# Setup

In [ ]:
# @title Cloning Static Analysis Tool Research Paper
!git clone https://github.com/Omar-Elsendiony/Static_Tool_Analysis_RP.git
%cd Static_Tool_Analysis_RP

Cloning into 'Static_Tool_Analysis_RP'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 59 (delta 24), reused 45 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 930.07 KiB | 3.03 MiB/s, done.
Resolving deltas: 100% (24/24), done.
/content/Static_Tool_Analysis_RP


In [ ]:
# @title Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Vulnerability Files

# Fixed Files

In [ ]:
# @title File-Only Version for attaining vulnerability
import os
import subprocess
import shutil
import json
import re
from collections import defaultdict

def analyze_commit_changes(repo_path, commit_id):
    """
    Analyze changes made in a specific commit compared to its parent.

    :param repo_path: Path to the local git repository
    :param commit_id: Commit ID to analyze
    :return: Dictionary containing detailed information about the changes
    """
    try:
        # Ensure we're at the right commit
        subprocess.run(["git", "-C", repo_path, "fetch", "origin", commit_id],
                check=True, capture_output=True)
        subprocess.run(["git", "-C", repo_path, "checkout", commit_id],
                check=True, capture_output=True)

        # Get commit details
        commit_info = subprocess.run(
            ["git", "-C", repo_path, "show", "--quiet", "--pretty=format:%an|%ae|%at|%s", commit_id],
            check=True, capture_output=True, text=True
        ).stdout.strip().split("|")

        # Get list of files changed
        files_changed = subprocess.run(
            ["git", "-C", repo_path, "diff-tree", "--no-commit-id", "--name-only", "-r", commit_id],
            check=True, capture_output=True, text=True
        ).stdout.strip().split("\n")

        # Get stats summary
        stats_summary = subprocess.run(
            ["git", "-C", repo_path, "diff", "--stat", f"{commit_id}^", commit_id],
            check=True, capture_output=True, text=True
        ).stdout.strip()

        # Get detailed diff
        diff_output = subprocess.run(
            ["git", "-C", repo_path, "diff", f"{commit_id}^", commit_id],
            check=True, capture_output=True, text=True
        ).stdout

        # Parse diff to count additions and deletions by file
        file_stats = defaultdict(lambda: {"additions": 0, "deletions": 0, "changes": 0})
        current_file = None

        for line in diff_output.split("\n"):
            if line.startswith("diff --git"):
                # Extract the file name from the diff header
                file_path = line.split(" b/")[-1]
                current_file = file_path
            elif line.startswith("+") and not line.startswith("+++"):
                file_stats[current_file]["additions"] += 1
                file_stats[current_file]["changes"] += 1
            elif line.startswith("-") and not line.startswith("---"):
                file_stats[current_file]["deletions"] += 1
                file_stats[current_file]["changes"] += 1

        # Calculate totals
        total_additions = sum(stats["additions"] for stats in file_stats.values())
        total_deletions = sum(stats["deletions"] for stats in file_stats.values())
        total_changes = total_additions + total_deletions

        # Determine file types changed
        file_extensions = defaultdict(int)
        for file in files_changed:
            if "." in file:
                ext = file.split(".")[-1].lower()
                file_extensions[ext] += 1
            else:
                file_extensions["no_extension"] += 1

        # Parse the diff to identify changed methods in Python files
        # changed_methods = extract_changed_methods_from_diff(diff_output)

        # Assemble results
        result = {
            "files": {
                file: stats for file, stats in file_stats.items()
            },
            "files_list": files_changed,
            "diff_summary": stats_summary,
            "full_diff": diff_output,
            # "changed_methods": changed_methods
        }

        return result

    except subprocess.CalledProcessError as e:
        print(f"Git command error: {e}")
        return {"error": str(e), "stdout": e.stdout, "stderr": e.stderr}
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return {"error": str(e)}

#     return result



def save_analysis_to_file(analysis, output_file):
    """
    Save the analysis results to a JSON file

    :param analysis: Analysis results dictionary
    :param output_file: Path to output file
    """
    with open(output_file, 'w') as f:
        json.dump(analysis, f, indent=2)
    print(f"Analysis saved to {output_file}")

def clone_repo(repo_owner, repo_name, repo_playground):
    if (repo_name) in os.listdir(repo_playground):
        return
    os.makedirs(repo_name)
    try:
        print(
            f"Cloning repository from https://github.com/{repo_owner}/{repo_name}.git to {repo_playground}/{repo_name}..."
        )
        subprocess.run(
            [
                "git",
                "clone",
                f"https://github.com/{repo_owner}/{repo_name}.git",
                f"{repo_playground}/{repo_name}",
            ],
            check=True,
        )
        print("Repository cloned successfully.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while running git command: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def postprocessing(analysis, repo_name, repo_path, commit_id):
    """
    Process and save both entire files and changed methods from a commit.

    :param analysis: Analysis results dictionary
    :param repo_name: Name of the repository
    :param repo_path: Path to the local repository
    :param commit_id: Commit ID being analyzed
    """
    # Filter Python files, excluding test files
    vul_files_fix = []
    for file_path in analysis['files_list']:
        if ".py" not in file_path:
            continue
        vul_files_fix.append(file_path)

    # Define base paths
    BASE_DIR = "repos_vulnerability_analysis"
    REPO_DIR = os.path.join(BASE_DIR, repo_name)
    COMMIT_DIR = os.path.join(REPO_DIR, commit_id[:7])
    FILES_DIR = os.path.join(COMMIT_DIR, "files")
    # METHODS_DIR = os.path.join(COMMIT_DIR, "methods")

    # Create directory structure
    for directory in [BASE_DIR, REPO_DIR, COMMIT_DIR, FILES_DIR]:
        os.makedirs(directory, exist_ok=True)

    metadata = {
    # Save metadata about the commit
        "commit_id": commit_id,
        "repo_name": repo_name,
        "files_changed": vul_files_fix,
        "changed_methods": analysis.get('changed_methods', {})
    }

    # with open(os.path.join(COMMIT_DIR, "metadata.json"), 'w') as f:
    #     json.dump(metadata, f, indent=2)

    # Save entire files
    for file_path in vul_files_fix:
        src_path = os.path.join(repo_path, file_path)
        if not os.path.exists(src_path):
            print(f"File not found: {src_path}")
            continue

        # Create a filename based on the original file's basename
        dst_filename = os.path.basename(file_path)
        dst_path = os.path.join(FILES_DIR, dst_filename)

        # Copy the entire file
        shutil.copyfile(src_path, dst_path)
        print(f"Copied file {file_path} to {dst_path}")

    # Extract and save changed methods
    # for file_path in vul_files_fix:
    #     # print(analysis.get('changed_methods'))
    #     if file_path not in analysis.get('changed_methods', {}):
    #         continue

    #     full_file_path = os.path.join(repo_path, file_path)
    #     if not os.path.exists(full_file_path):
    #         print(f"File not found: {full_file_path}")
    #         continue


# Example usage
if __name__ == "__main__":
    with open('repos_fixes/filtered_fixes.json', 'r') as f:
        repos_fixes = json.load(f)

    # path of the folder where the repositories will be cloned
    repo_playground = "."

    # constants to access elements in the repos_fixes list
    REPO_URL_INDEX = 2
    COMMIT_ID_INDEX = 1

    number = 0

    for repo in repos_fixes:
        repo_owner, repo_name = repo[REPO_URL_INDEX].split("/")[-2:]
        # repo_owner, repo_name = "zopefoundation", "zope"
        commit_id = repo[COMMIT_ID_INDEX]
        repo_path = os.path.join(".", repo_name)

        # Clone the repository
        clone_repo(repo_owner, repo_name, repo_playground)

        # Analyze the commit
        analysis = analyze_commit_changes(repo_path, commit_id)
        # There should be no error in the analysis
        if "error" in analysis:
            print(f"Error analyzing commit: {analysis['error']}")
            continue


        python_files_found = 0
        for file_path in analysis['files_list']:
            src_path = os.path.join(repo_path, file_path)
            if not os.path.exists(src_path) or ".py" not in src_path:
                continue
            python_files_found += 1

        if (python_files_found == 0):
            print("No Python Files Changed")
            continue

        stash_command = ["git", "-C", repo_path, "stash"]

        stash_result = subprocess.run(stash_command,
                              check=False,
                              capture_output=True,
                              text=True,
                              encoding='utf-8')

        # Checkout to the old version before the patch
        subprocess.run(["git", "-C", repo_path, "checkout", "HEAD^"],
            check=True, capture_output=True)


        postprocessing(analysis, repo_name, repo_path, commit_id)

        number += 1
        print(f"Processed repository {number}: {repo_name}, commit {commit_id[:7]}")

        # if (number == 50):
        #   break

Copied file src/Products/PageTemplates/ZRPythonExpr.py to repos_vulnerability_analysis/Zope/2abdf14/files/ZRPythonExpr.py
Copied file src/Products/PageTemplates/tests/testZRPythonExpr.py to repos_vulnerability_analysis/Zope/2abdf14/files/testZRPythonExpr.py
Processed repository 1: Zope, commit 2abdf14
Copied file Pyro/ext/daemonizer.py to repos_vulnerability_analysis/Pyro3/554e095/files/daemonizer.py
Processed repository 2: Pyro3, commit 554e095
Copied file src/lib/Server/Admin/Viz.py to repos_vulnerability_analysis/bcfg2/f4a35ef/files/Viz.py
Copied file src/lib/Server/Plugins/Hg.py to repos_vulnerability_analysis/bcfg2/f4a35ef/files/Hg.py
Copied file src/lib/Server/Plugins/SSHbase.py to repos_vulnerability_analysis/bcfg2/f4a35ef/files/SSHbase.py
Copied file src/lib/Server/Plugins/SSLCA.py to repos_vulnerability_analysis/bcfg2/f4a35ef/files/SSLCA.py
Copied file src/lib/Server/Plugins/Svn.py to repos_vulnerability_analysis/bcfg2/f4a35ef/files/Svn.py
Processed repository 3: bcfg2, commit

In [ ]:
# @title File-Only Version for acquiring fixes
import os
import subprocess
import shutil
import json
import re
from collections import defaultdict

def analyze_commit_changes(repo_path, commit_id):
    """
    Analyze changes made in a specific commit compared to its parent.

    :param repo_path: Path to the local git repository
    :param commit_id: Commit ID to analyze
    :return: Dictionary containing detailed information about the changes
    """
    try:
        # Ensure we're at the right commit
        subprocess.run(["git", "-C", repo_path, "fetch", "origin", commit_id],
                check=True, capture_output=True)
        subprocess.run(["git", "-C", repo_path, "checkout", commit_id],
                check=True, capture_output=True)

        # Get commit details
        commit_info = subprocess.run(
            ["git", "-C", repo_path, "show", "--quiet", "--pretty=format:%an|%ae|%at|%s", commit_id],
            check=True, capture_output=True, text=True
        ).stdout.strip().split("|")

        # Get list of files changed
        files_changed = subprocess.run(
            ["git", "-C", repo_path, "diff-tree", "--no-commit-id", "--name-only", "-r", commit_id],
            check=True, capture_output=True, text=True
        ).stdout.strip().split("\n")

        # Get stats summary
        stats_summary = subprocess.run(
            ["git", "-C", repo_path, "diff", "--stat", f"{commit_id}^", commit_id],
            check=True, capture_output=True, text=True
        ).stdout.strip()

        # Get detailed diff
        diff_output = subprocess.run(
            ["git", "-C", repo_path, "diff", f"{commit_id}^", commit_id],
            check=True, capture_output=True, text=True
        ).stdout

        # Parse diff to count additions and deletions by file
        file_stats = defaultdict(lambda: {"additions": 0, "deletions": 0, "changes": 0})
        current_file = None

        for line in diff_output.split("\n"):
            if line.startswith("diff --git"):
                # Extract the file name from the diff header
                file_path = line.split(" b/")[-1]
                current_file = file_path
            elif line.startswith("+") and not line.startswith("+++"):
                file_stats[current_file]["additions"] += 1
                file_stats[current_file]["changes"] += 1
            elif line.startswith("-") and not line.startswith("---"):
                file_stats[current_file]["deletions"] += 1
                file_stats[current_file]["changes"] += 1

        # Calculate totals
        total_additions = sum(stats["additions"] for stats in file_stats.values())
        total_deletions = sum(stats["deletions"] for stats in file_stats.values())
        total_changes = total_additions + total_deletions

        # Determine file types changed
        file_extensions = defaultdict(int)
        for file in files_changed:
            if "." in file:
                ext = file.split(".")[-1].lower()
                file_extensions[ext] += 1
            else:
                file_extensions["no_extension"] += 1

        # Parse the diff to identify changed methods in Python files
        # changed_methods = extract_changed_methods_from_diff(diff_output)

        # Assemble results
        result = {
            "files": {
                file: stats for file, stats in file_stats.items()
            },
            "files_list": files_changed,
            "diff_summary": stats_summary,
            "full_diff": diff_output,
            # "changed_methods": changed_methods
        }

        return result

    except subprocess.CalledProcessError as e:
        print(f"Git command error: {e}")
        return {"error": str(e), "stdout": e.stdout, "stderr": e.stderr}
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return {"error": str(e)}

#     return result



def save_analysis_to_file(analysis, output_file):
    """
    Save the analysis results to a JSON file

    :param analysis: Analysis results dictionary
    :param output_file: Path to output file
    """
    with open(output_file, 'w') as f:
        json.dump(analysis, f, indent=2)
    print(f"Analysis saved to {output_file}")

def clone_repo(repo_owner, repo_name, repo_playground):
    if (repo_name) in os.listdir(repo_playground):
        return
    os.makedirs(repo_name)
    try:
        print(
            f"Cloning repository from https://github.com/{repo_owner}/{repo_name}.git to {repo_playground}/{repo_name}..."
        )
        subprocess.run(
            [
                "git",
                "clone",
                f"https://github.com/{repo_owner}/{repo_name}.git",
                f"{repo_playground}/{repo_name}",
            ],
            check=True,
        )
        print("Repository cloned successfully.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while running git command: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def postprocessing(analysis, repo_name, repo_path, commit_id):
    """
    Process and save both entire files and changed methods from a commit.

    :param analysis: Analysis results dictionary
    :param repo_name: Name of the repository
    :param repo_path: Path to the local repository
    :param commit_id: Commit ID being analyzed
    """
    # Filter Python files, excluding test files
    vul_files_fix = []
    for file_path in analysis['files_list']:
        if ".py" not in file_path:
            continue
        vul_files_fix.append(file_path)

    # Define base paths
    BASE_DIR = "repos_patched_analysis"
    REPO_DIR = os.path.join(BASE_DIR, repo_name)
    COMMIT_DIR = os.path.join(REPO_DIR, commit_id[:7])
    FILES_DIR = os.path.join(COMMIT_DIR, "files")
    # METHODS_DIR = os.path.join(COMMIT_DIR, "methods")

    # Create directory structure
    for directory in [BASE_DIR, REPO_DIR, COMMIT_DIR, FILES_DIR]:
        os.makedirs(directory, exist_ok=True)

    metadata = {
    # Save metadata about the commit
        "commit_id": commit_id,
        "repo_name": repo_name,
        "files_changed": vul_files_fix,
        "changed_methods": analysis.get('changed_methods', {})
    }

    # with open(os.path.join(COMMIT_DIR, "metadata.json"), 'w') as f:
    #     json.dump(metadata, f, indent=2)

    # Save entire files
    for file_path in vul_files_fix:
        src_path = os.path.join(repo_path, file_path)
        if not os.path.exists(src_path):
            print(f"File not found: {src_path}")
            continue

        # Create a filename based on the original file's basename
        dst_filename = os.path.basename(file_path)
        dst_path = os.path.join(FILES_DIR, dst_filename)

        # Copy the entire file
        shutil.copyfile(src_path, dst_path)
        print(f"Copied file {file_path} to {dst_path}")

    # Extract and save changed methods
    # for file_path in vul_files_fix:
    #     # print(analysis.get('changed_methods'))
    #     if file_path not in analysis.get('changed_methods', {}):
    #         continue

    #     full_file_path = os.path.join(repo_path, file_path)
    #     if not os.path.exists(full_file_path):
    #         print(f"File not found: {full_file_path}")
    #         continue


# Example usage
if __name__ == "__main__":
    with open('repos_fixes/filtered_fixes.json', 'r') as f:
        repos_fixes = json.load(f)

    # path of the folder where the repositories will be cloned
    repo_playground = "."

    # constants to access elements in the repos_fixes list
    REPO_URL_INDEX = 2
    COMMIT_ID_INDEX = 1

    number = 0

    for repo in repos_fixes:
        repo_owner, repo_name = repo[REPO_URL_INDEX].split("/")[-2:]
        # repo_owner, repo_name = "zopefoundation", "zope"
        commit_id = repo[COMMIT_ID_INDEX]
        repo_path = os.path.join(".", repo_name)

        # Clone the repository
        clone_repo(repo_owner, repo_name, repo_playground)

        # Analyze the commit
        analysis = analyze_commit_changes(repo_path, commit_id)
        # There should be no error in the analysis
        if "error" in analysis:
            print(f"Error analyzing commit: {analysis['error']}")
            continue


        python_files_found = 0
        for file_path in analysis['files_list']:
            src_path = os.path.join(repo_path, file_path)
            if not os.path.exists(src_path) or ".py" not in src_path:
                continue
            python_files_found += 1

        if (python_files_found == 0):
            print("No Python Files Changed")
            continue

        stash_command = ["git", "-C", repo_path, "stash"]

        stash_result = subprocess.run(stash_command,
                              check=False,
                              capture_output=True,
                              text=True,
                              encoding='utf-8')

        # NOOOOOOOOOOOOO CHECKOUTTTTTTTTT
        # subprocess.run(["git", "-C", repo_path, "checkout", "HEAD^"],
        #     check=True, capture_output=True)


        postprocessing(analysis, repo_name, repo_path, commit_id)

        number += 1
        print(f"Processed repository {number}: {repo_name}, commit {commit_id[:7]}")

        # if (number == 50):
        #   break

Cloning repository from https://github.com/zopefoundation/Zope.git to ./Zope...
Repository cloned successfully.
Copied file src/Products/PageTemplates/ZRPythonExpr.py to repos_vulnerability_analysis/Zope/2abdf14/files/ZRPythonExpr.py
Copied file src/Products/PageTemplates/tests/testZRPythonExpr.py to repos_vulnerability_analysis/Zope/2abdf14/files/testZRPythonExpr.py
Processed repository 1: Zope, commit 2abdf14
Cloning repository from https://github.com/irmen/Pyro3.git to ./Pyro3...
Repository cloned successfully.
Copied file Pyro/ext/daemonizer.py to repos_vulnerability_analysis/Pyro3/554e095/files/daemonizer.py
Processed repository 2: Pyro3, commit 554e095
Cloning repository from https://github.com/solj/bcfg2.git to ./bcfg2...
Repository cloned successfully.
Copied file src/lib/Server/Admin/Viz.py to repos_vulnerability_analysis/bcfg2/f4a35ef/files/Viz.py
Copied file src/lib/Server/Plugins/Hg.py to repos_vulnerability_analysis/bcfg2/f4a35ef/files/Hg.py
Copied file src/lib/Server/Plug

In [ ]:
# @title Save to Google Drive
!zip repos_patched_analysis.zip -r repos_patched_analysis
!mv repos_patched_analysis.zip /content/drive/MyDrive/

  adding: repos_patched_analysis/ (stored 0%)
  adding: repos_patched_analysis/guarddog/ (stored 0%)
  adding: repos_patched_analysis/guarddog/37c7d07/ (stored 0%)
  adding: repos_patched_analysis/guarddog/37c7d07/files/ (stored 0%)
  adding: repos_patched_analysis/guarddog/37c7d07/files/package_scanner.py (deflated 72%)
  adding: repos_patched_analysis/weblate/ (stored 0%)
  adding: repos_patched_analysis/weblate/abe0d2a/ (stored 0%)
  adding: repos_patched_analysis/weblate/abe0d2a/files/ (stored 0%)
  adding: repos_patched_analysis/weblate/abe0d2a/files/views.py (deflated 73%)
  adding: repos_patched_analysis/weblate/abe0d2a/files/test_registration.py (deflated 78%)
  adding: repos_patched_analysis/weblate/22d577b/ (stored 0%)
  adding: repos_patched_analysis/weblate/22d577b/files/ (stored 0%)
  adding: repos_patched_analysis/weblate/22d577b/files/reports.py (deflated 75%)
  adding: repos_patched_analysis/weblate/22d577b/files/test_reports.py (deflated 78%)
  adding: repos_patched_an

In [ ]:
# @title Golden Truth

import os
import subprocess
import json

def postprocessing(repo_name, cve_id, cve_cwe_mapping):
    ####################### POSTPROCESSING ########################
    # valid_files_fix = []
    # for file_path, _ in analysis['files'].items():
    #     if ".rst"  in file_path or ".md"  in file_path or "test" in file_path:
    #         continue
    #     valid_files_fix.append(file_path)

    BASE_REPOS_EDITED_FILES_CONTENT_PATH = "Golden_Truth"
    SPECIFIC_REPOS_EDITED_FILES_CONTENT_PATH = os.path.join(BASE_REPOS_EDITED_FILES_CONTENT_PATH, repo_name)
    SPECIFIC_COMMIT_EDITED_FILES_CONTENT_PATH = os.path.join(SPECIFIC_REPOS_EDITED_FILES_CONTENT_PATH, commit_id[:7])
    if not os.path.exists(BASE_REPOS_EDITED_FILES_CONTENT_PATH):
        os.makedirs(BASE_REPOS_EDITED_FILES_CONTENT_PATH, exist_ok=True)
    if not os.path.exists(SPECIFIC_REPOS_EDITED_FILES_CONTENT_PATH):
        os.makedirs(SPECIFIC_REPOS_EDITED_FILES_CONTENT_PATH, exist_ok=True)
    # if not os.path.exists(os.path.join(BASE_REPOS_EDITED_FILES_CONTENT_PATH, repo_name, commit_id[:7])):
    os.makedirs(SPECIFIC_COMMIT_EDITED_FILES_CONTENT_PATH, exist_ok=True)

    my_dict = {"cve_id": cve_id, "cwe_id": cve_cwe_mapping[cve_id]}

    try:
        with open(os.path.join(SPECIFIC_COMMIT_EDITED_FILES_CONTENT_PATH, "golden_truth.json"), 'w') as json_file:
            json.dump(my_dict, json_file, indent=4)  # Use indent for pretty formatting
        print(f"Data written to {SPECIFIC_COMMIT_EDITED_FILES_CONTENT_PATH}")
    except Exception as e:
        print(f"An error occurred while writing to the JSON file: {e}")

# Example usage
if __name__ == "__main__":

    with open('cve_cwe_mapping/cve_cwe_mappings_dict.json', 'r') as f:
        cve_cwe_mapping = json.load(f)

    with open('repos_fixes/filtered_fixes.json', 'r') as f:
        repos_fixes = json.load(f)


    # constants to access elements in the repos_fixes list
    REPO_URL_INDEX = 2
    COMMIT_ID_INDEX = 1
    CVE_ID_INDEX = 0

    number = 0

    for repo in repos_fixes:
        repo_owner, repo_name = repo[REPO_URL_INDEX].split("/")[-2:]
        commit_id = repo[COMMIT_ID_INDEX]
        cve_id = repo[CVE_ID_INDEX]

        postprocessing(repo_name, cve_id, cve_cwe_mapping)

        # number += 1
        # if (number == 20):
        #     break


Data written to Golden_Truth/Zope/2abdf14
Data written to Golden_Truth/Pyro3/554e095
Data written to Golden_Truth/bcfg2/f4a35ef
Data written to Golden_Truth/reviewboard/7a0a9d9
Data written to Golden_Truth/nova/ad32419
Data written to Golden_Truth/nova/1f644d2
Data written to Golden_Truth/pycrypto/9f912f1
Data written to Golden_Truth/nova/ff06c7c
Data written to Golden_Truth/nova/2427d4a
Data written to Golden_Truth/nova/2427d4a
Data written to Golden_Truth/bcfg2/a524967
Data written to Golden_Truth/nova/034762e
Data written to Golden_Truth/keystone/375838c
Data written to Golden_Truth/keystone/628149b
Data written to Golden_Truth/keystone/a67b248
Data written to Golden_Truth/nova/ce4b2e2
Data written to Golden_Truth/beaker/91becae
Data written to Golden_Truth/keystone/c13d0ba
Data written to Golden_Truth/swift/e1ff51c
Data written to Golden_Truth/keystone/1d146f5
Data written to Golden_Truth/keystone/8680549
Data written to Golden_Truth/keystone/4ebfdfa
Data written to Golden_Truth/dj

In [ ]:
!rm -r Golden_Truth

In [ ]:
# @title Save Golden Truth to Google Drive

!zip Golden_Truth.zip -r Golden_Truth
!mv Golden_Truth.zip /content/drive/MyDrive/

  adding: Golden_Truth/ (stored 0%)
  adding: Golden_Truth/calibre/ (stored 0%)
  adding: Golden_Truth/calibre/aeb5b03/ (stored 0%)
  adding: Golden_Truth/calibre/aeb5b03/golden_truth.json (deflated 27%)
  adding: Golden_Truth/ncpa/ (stored 0%)
  adding: Golden_Truth/ncpa/5abbcd7/ (stored 0%)
  adding: Golden_Truth/ncpa/5abbcd7/golden_truth.json (deflated 27%)
  adding: Golden_Truth/vyper/ (stored 0%)
  adding: Golden_Truth/vyper/2c73f83/ (stored 0%)
  adding: Golden_Truth/vyper/2c73f83/golden_truth.json (deflated 27%)
  adding: Golden_Truth/vyper/049dbdc/ (stored 0%)
  adding: Golden_Truth/vyper/049dbdc/golden_truth.json (deflated 27%)
  adding: Golden_Truth/vyper/3de1415/ (stored 0%)
  adding: Golden_Truth/vyper/3de1415/golden_truth.json (deflated 27%)
  adding: Golden_Truth/vyper/55e18f6/ (stored 0%)
  adding: Golden_Truth/vyper/55e18f6/golden_truth.json (deflated 27%)
  adding: Golden_Truth/vyper/851f7a1/ (stored 0%)
  adding: Golden_Truth/vyper/851f7a1/golden_truth.json (deflated 

In [ ]:
%pwd

'/content/Static_Tool_Analysis_RP'